In [1]:
import pandas as pd
import numpy as np
import scipy as scp
import geopy # pip install geopy if necessary

In [2]:
data = pd.read_csv('../openfoodfacts.csv',delimiter='\t',low_memory=False)

In [3]:
print(data.shape)

(681602, 173)


# Data for distance computation

In [4]:
#a = ~data["first_packaging_code_geo"].isnull() # 27572
b = ~data["origins_tags"].isnull()
c = ~data["manufacturing_places_tags"].isnull()

sele = b | c  # 88001

data_loc = data[sele] # 57688 (88001)

sel = ['categories_en', 'main_category_en', 'labels_en', 'origins_tags', 'manufacturing_places_tags', 
       'first_packaging_code_geo', 'countries_en']

data_loc = data_loc[sel]
data_loc

,categories_en,main_category_en,labels_en,origins_tags,manufacturing_places_tags,first_packaging_code_geo,countries_en
106,Dietary supplements,Dietary supplements,Made in France,NaN,france,NaN,France
251,"Meals,Meat-based products,Meals with meat,Poul...",Meals,"fr:Viande Française,Made in France",france,france,NaN,France
254,"Plant-based foods and beverages,Plant-based fo...",Plant-based foods and beverages,NaN,quebec,brossard-quebec,NaN,Canada
264,"Sugary snacks,Biscuits and cakes,Pastries",Sugary snacks,NaN,quebec,brossard-quebec,NaN,Canada
269,"Plant-based foods and beverages,Plant-based fo...",Plant-based foods and beverages,NaN,france,united-kingdom,NaN,United Kingdom
274,"Sugary snacks,Biscuits and cakes,Pastries",Sugary snacks,NaN,quebec,brossard-quebec,NaN,Canada
285,fr:Boulange,fr:Boulange,NaN,quebec,"brossard,quebec",NaN,Canada
289,"Beverages,Carbonated drinks,Sodas,Sugared beve...",Beverages,"Kosher,Contains GMOs",NaN,etats-unis,NaN,"France,United States"
292,"Sugary snacks,Biscuits and cakes,Biscuits,Shor...",Sugary snacks,Green Dot,NaN,"france,avranches",NaN,France
296,"Meals,Pizzas pies and quiches,Quiches,Lorraine...",Meals,NaN,quebec,"brossard,quebec",NaN,Canada


## Pre-process on coutrnies_en section

Sometimes the countries_en section contains several countries. Thus duplications of the product have to be done in order to consider the product in each country independantly.

In [5]:
z = data_loc["countries_en"].str.split(',').apply(pd.Series)
z1 = z.unstack().dropna()
z1 = z1.to_frame()

In [6]:
z1.columns = ['destination']

In [7]:
# change destination with special characters
# manually checked on wikipedia + google maps
z1[z1['destination'] == 'Other-日本'] = 'Japon' # manually checked on wiki
z1[z1['destination'] == '中国'] = 'Chine'
z1[z1['destination'] == 'fr:香港'] = 'Hong Kong'
z1[z1['destination'] == 'fr:日本'] = 'Japon'
z1[z1['destination'] == 'Ελλάδα'] = 'Grèce'
z1[z1['destination'] == 'ar:صنعاء'] = 'Sanaa'
z1[z1['destination'] == 'fr:الجزائر'] = 'Algérie'


In [8]:
# remove fr: or others xx: in destination
def split_xx(l):
    tmp = l.split(':')
    
    if len(tmp) == 1:
        return l
    else:
        return tmp[1]
        
    #.str.get(0).str[1,:]
        
z1['destination'] = z1['destination'].apply(lambda l: split_xx(l))

In [9]:
print(z1.shape)
z1 = z1[~(z1 == 'Photos uploaded')]
z1 = z1[~(z1 == 'Photos validated')]
z1 = z1[~(z1 == 'Product name completed')]
z1 = z1[~(z1 == 'Photos to be uploaded')]
z1 = z1[~(z1 == 'Photos to be validated')]
z1 = z1[~(z1 == 'Quantity completed')]
z1 = z1[~(z1 == 'Nutrition facts completed')]
z1 = z1[~(z1 == 'Ingredients completed')]
z1 = z1[~(z1 == 'Brands completed')]
z1 = z1[~(z1 == 'Packaging completed')]
z1 = z1[~(z1 == 'Characteristics completed')]
z1 = z1[~(z1 == 'Categories completed')]
z1 = z1[~(z1 == 'Packaging-code-completed')]
z1 = z1[~(z1 == 'Expiration date completed')]
z1 = z1[~(z1 == 'Expiration date to be completed')]
z1 = z1[~(z1 == 'En')]
z1 = z1[~(z1 == 'To be completed')]
z1 = z1[~(z1 == 'To be checked')]
z1 = z1[~(z1 == 'Worldwide')]
z1 = z1[~(z1 == 'World')]
z1 = z1[~(z1 == 'Sucre')]
z1 = z1[~(z1 == 'Complete')]
z1 = z1[~(z1 == 'Global-market')]
z1 = z1[~(z1 == 'Dat')]
z1 = z1[~(z1 == 'France-others')]

z1 = z1.dropna()
print(z1.shape)
#z1 = z1.to_frame()

(89510, 1)
(88959, 1)


In [10]:
# replace values manually
z1[z1['destination'] == 'Leclerc-bois-d-arcy-france'] = 'France'
z1[z1['destination'] == 'Francecontient-des-sulfites'] = 'France'
z1[z1['destination'] == 'Leclerc'] = 'France'
z1[z1['destination'] == 'Auchan'] = 'France'
z1[z1['destination'] == 'Carrefour'] = 'France'
z1[z1['destination'] == 'Estadps-unidos'] = 'United States'
z1[z1['destination'] == 'Polyensie-francaise'] = 'French Polynesia'

In [12]:
z1 = z1.swaplevel()
z1 = z1.reset_index(level=1, drop=True)

In [13]:
z1.head()

,destination
106,France
251,France
254,Canada
264,Canada
269,United Kingdom


Apply the dictionnary to translate all countries in english

## Select french products

In [15]:
countries = pd.read_csv('data/Destination-Coordinate-MAN.csv',delimiter=',',low_memory=False)
dico_countries = dict(zip(countries['destination'], countries['country']))

In [16]:
dico_countries

{'France': 'France',
 'Spain': 'Spain',
 'Germany': 'Germany',
 'Switzerland': 'Switzerland',
 'United Kingdom': 'United Kingdom',
 'Belgium': 'Belgium',
 'Mexico': 'Mexico',
 'Australia': 'Australia',
 'United States': 'USA',
 'Italy': 'Italy',
 'Sweden': 'Sweden',
 'Canada': 'Canada',
 'Serbia': 'Serbia',
 'Portugal': 'Portugal',
 'Russia': 'Russia',
 'Romania': 'Romania',
 'Netherlands': 'Netherlands',
 'Austria': 'Austria',
 'Denmark': 'Denmark',
 'Réunion': 'France',
 'Hungary': 'Hungary',
 'Guadeloupe': 'France',
 'Brazil': 'Brazil',
 'French Polynesia': 'France',
 'Hong Kong': 'Hong Kong',
 'Poland': 'Poland',
 'Madagascar': 'Madagascar',
 'Ireland': 'Ireland',
 'Tunisia': 'Tunisia',
 'New Zealand': 'New Zealand',
 'India': 'India',
 'South Africa': 'RSA',
 'Luxembourg': 'Luxembourg',
 'French Guiana': 'France',
 'Georgia': 'USA',
 'Deutschland': 'Germany',
 'Czech Republic': 'Czechia',
 'Morocco': 'Morocco',
 'Algeria': 'Algeria',
 'Martinique': 'France',
 'Greece': 'Greece',
 

In [20]:
z1['Arrival'] = z1['destination'].apply(lambda l: dico_countries[l])
z1.head()

,destination,Arrival
106,France,France
251,France,France
254,Canada,Canada
264,Canada,Canada
269,United Kingdom,United Kingdom


In [21]:
selection = z1[z1['Arrival'] == 'France']

In [23]:
selection.shape

(61884, 2)

In [24]:
selection.head()

,destination,Arrival
106,France,France
251,France,France
289,France,France
292,France,France
299,France,France


In [28]:
selection.index

Int64Index([   106,    251,    289,    292,    299,    305,    356,    410,
               526,    582,
            ...
            662589, 676662, 676663, 676664, 101991, 342121, 498576, 567379,
            675417, 675397],
           dtype='int64', length=61884)

In [47]:
french_product = data_loc.loc[selection.index]
french_product.head()

,categories_en,main_category_en,labels_en,origins_tags,manufacturing_places_tags,first_packaging_code_geo,countries_en
106,Dietary supplements,Dietary supplements,Made in France,NaN,france,NaN,France
251,"Meals,Meat-based products,Meals with meat,Poul...",Meals,"fr:Viande Française,Made in France",france,france,NaN,France
289,"Beverages,Carbonated drinks,Sodas,Sugared beve...",Beverages,"Kosher,Contains GMOs",NaN,etats-unis,NaN,"France,United States"
292,"Sugary snacks,Biscuits and cakes,Biscuits,Shor...",Sugary snacks,Green Dot,NaN,"france,avranches",NaN,France
299,"Sugary snacks,Confectioneries,Candies",Sugary snacks,"No artificial flavors,Contains GMOs",NaN,etats-unis,NaN,France


In [48]:
french_product.to_csv('data/FrenchProduct.csv')

In [52]:
# check
country_count = french_product["countries_en"].value_counts()[0:20]
country_count

France                        54889
France,Switzerland             1225
Belgium,France                  706
France,Germany                  630
France,Spain                    604
France,United Kingdom           596
France,Réunion                  216
France,Guadeloupe               184
France,Italy                    183
France,United States            156
France,Germany,Switzerland      119
France,French Polynesia         118
France,French Guiana            102
Australia,France                 66
France,Martinique                66
France,Netherlands               63
Canada,France                    61
French Polynesia                 58
France,Portugal                  57
Belgium,France,Switzerland       57
Name: countries_en, dtype: int64